In [1]:
from AHI_AC import *

# Set spatial resolution
# 0.01deg ==> 0.01, 0.02deg ==> 0.02
res = 0.02

# Set latitude upper and bottom bounds
lat_up, lat_bottom = 60, -60

# Set longitude left and right bounds
lon_left, lon_right = 85, 205

# Set save folder
target = '/data01/GEO/SRDATA/'

# See main.py
row,col,row_AHI,col_AHI,AHI_lat,AHI_lon,AHI_VZA, AHI_VAA,AHI_AL,Water_idx,Landmask = main_read(res,lat_up, lat_bottom,lon_left, lon_right)

# Read LUT and interpolate with LUT
# Processing band can be set by yourself!

interpolation = LUT_interpolator(5)
fn1_5, fn2_5, fn3_5 = interpolation._interpolate()
interpolation = LUT_interpolator(6)
fn1_6, fn2_6, fn3_6 = interpolation._interpolate()

# Set necessary parameters
band_type = {'5': 'sir', '6': 'sir', }
band_num = {'5': '01', '6': '02'}
calculate_6s = {'5': calculate_6s_band5, '6': calculate_6s_band6}
fn1 = {'5': fn1_5, '6': fn1_6}
fn2 = {'5': fn2_5, '6': fn2_6}
fn3 = {'5': fn3_5, '6': fn3_6}


In [2]:
def SR_Process(band):
    start_time = T.time()

    # Download AHI
    print(f"Start download AHI for Band{band}")
    AHI_data = H8_Process('ACCOUNT', 'PW', band_type[band], band_num[band], date, folder_original)
    if AHI_data == 'No data':
        print(f"No AHI data for Band{band}, pass")
        return None
    E_Read_AHI_time = T.time()
    print(f"Finish download AHI for Band{band}")

    # SR calculate
    print(f'Start Calculate SR for Band{band}')
    S_SR_time = T.time()
    RESULT = Parallel(n_jobs=njob)(delayed(calculate_6s[band])(fn1[band], fn2[band], fn3[band], Landmask, Aerosol_type, *params[band], AHI_data, i) for i in range(6000))
    SR = np.full((6000, 6000), np.nan)
    for i in range(6000):
        SR[i][Water_idx[i]] = RESULT[i]
    E_SR_time = T.time()
    print(f'Calculate SR for Band{band} finished')

    # Save file and remove download input data
    print(f'Start Save SR file for Band{band}')
    S_SAVE_time = T.time()
    SR = np.array(SR).reshape(6000, 6000)
    SR = np.around(SR, 4) * 10000

    SR = np.where(SR >= 32767, 32767, SR)
    SR = np.where(SR <= -32767, -32767, SR)
    SR = np.where(np.isnan(SR), -32768, SR)

    SR_file = open(folder_AC + f'H08_{{YYYY}}{{MM}}{{DD}}_{{HH}}{{MIN}}_B{band}_SR.dat', 'wb')
    SR.astype('int16').tofile(SR_file)
    SR_file.close()

    print(f'Save Band{band} SR finished')
    # remove_original_file(folder_original)
    # print("delete file finish")
    E_SAVE_time = T.time()
    end_time = T.time()
    TIME = end_time - start_time
    print('Band{} cost time: {:.1f} secs, {:.1f} mins\n'.format(band, TIME, TIME / 60))

In [3]:
njob = 15
date_start = '2019-07-31 00:10'
date_end = '2019-07-31 23:50'
date_t = dt.timedelta(minutes=30)
date_s = dt.datetime.strptime(date_start, "%Y-%m-%d %H:%M")
date_e = dt.datetime.strptime(date_end, "%Y-%m-%d %H:%M")
date_time_now = date_s
YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)
while date_time_now <= date_e:

    date_dl_str = date_time_now.strftime("%Y-%m-%d %H:%M" )
    YYYY,MM,DD,HH,MIN,date= Time_split(date_time_now)   
    hour = int(date[8:10])
    if hour >=2 and hour < 10:  
        S_time = T.time()
        print("Start processing {}".format(date))
    
        # make dir
        folder_original = f"{target}{date}_original/"
        folder_AC = f"{target}{YYYY}/{MM}/{DD}/"
        mkdir(folder_original)
        mkdir(folder_AC)

        # Solar angle
        print('Start read solar angle')
        S_ANGLE_time = T.time()
        AHI_SZA,AHI_SAA = AHI_angle(date,row_AHI).read_solar_angle()
        AHI_SZA,AHI_SAA = AHI_SZA[row:row+row_AHI,col:col+col_AHI],AHI_SAA[row:row+row_AHI,col:col+col_AHI]
        RAA = abs(AHI_SAA - AHI_VAA)
        RAA[RAA>180]=360-RAA[RAA>180]
        print('Finish read solar angle')
        E_ANGLE_time = T.time()
        print('Read angle cost : {}s'.format(round((E_ANGLE_time-S_ANGLE_time),4)))
        
        # Atmosphereic data
        print('Start read CAMS')
        S_ATMOS_time = T.time()
        CAMS_data_obj = CAMS_data(YYYY, MM, DD, HH, MIN, AHI_lat, AHI_lon, row_AHI)
        Aerosol_type, OZ, WV, AOT550 = read_cams_data(CAMS_data_obj)
        print('Finish read CAMS')
        E_ATMOS_time = T.time()
        print('Read Atmospheric data cost : {}s'.format(round((E_ATMOS_time-S_ATMOS_time),4)))

        params = {
                    '5': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL],
                    '6': [WV, AOT550, RAA, AHI_SZA, AHI_VZA, AHI_AL]
                }
        
        for band in ['5', '6']:
            SR_Process(band)

    else:
        print(f"{date_time_now} not in time range, pass")
    date_time_now = date_time_now + date_t

2019-07-31 00:10:00 not in time range, pass
2019-07-31 00:40:00 not in time range, pass
2019-07-31 01:10:00 not in time range, pass
2019-07-31 01:40:00 not in time range, pass
Start processing 201907310210
Start read solar angle
Finish read solar angle
Read angle cost : 0.3537s
Start read CAMS
Finish read CAMS
Read Atmospheric data cost : 1.3526s
Start download AHI for Band5
Finish download AHI for Band5
Start Calculate SR for Band5


<ipython-input-2-292b6278cc23>:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if AHI_data == 'No data':


Calculate SR for Band5 finished
Start Save SR file for Band5
Save Band5 SR finished
Band5 cost time: 17.6 secs, 0.3 mins

Start download AHI for Band6
Finish download AHI for Band6
Start Calculate SR for Band6


KeyboardInterrupt: 